In [1]:
import numpy as np
import scipy as sp
import xarray as xr
import matplotlib.pyplot as plt
import glob
import re
import cartopy.crs as ccrs  # Correct import for coordinate reference systems
import cartopy.feature as cfeature

In [2]:
import dask 
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from dask.distributed import performance_report

In [3]:
# Boulder Lat Lon
boulder_lat = 40.014
boulder_lon = -105.270 + 360
#
# Bounding box for Colorado
colorado_right  =  (-109.060253 + 360)%360
colorado_left   = (-102.041524 + 360)%360 
colorado_bottom = 37
colorado_top    = 41

In [4]:
# # File paths
# rda_scratch = '/gpfs/csfs1/collections/rda/scratch/harshah'
# rda_data    = '/gpfs/csfs1/collections/rda/data/'
# era5_path   = rda_data + 'ds633.0/e5.oper.an.sfc/'
# zarr_path   = rda_scratch + '/tas_zarr/'

In [5]:
# #Pelican test path
# pelican_test = '/glade/campaign/collections/rda/transfer/harshah/pelican_test'
# zarr_path   = rda_scratch + '/tas_zarr/'

In [6]:
# Use this path for https access
rda_url      =  'https://request.rda.ucar.edu/'
zarr_path    = rda_url + 'harshah/pelican_test/tas_zarr/'

In [7]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name = 'dask-wk24-hpc',
    cores = 1,
    memory = '8GiB',
    processes = 1,
    local_directory = '/dask/spill',
    resource_spec = 'select=1:ncpus=1:mem=8GB',
    queue = 'casper',
    walltime = '2:30:00',
    #interface = 'ib0'
    interface = 'ext'
)

/glade/work/harshah/conda-envs/zarr_experiments/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39555 instead
  warnings.warn(
Task exception was never retrieved
future: <Task finished name='Task-579' coro=<Client._gather.<locals>.wait() done, defined at /glade/work/harshah/conda-envs/zarr_experiments/lib/python3.11/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/glade/work/harshah/conda-envs/zarr_experiments/lib/python3.11/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit


In [8]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/39555/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/39555/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.119:41327,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/39555/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [9]:
cluster.scale(2)

In [10]:
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/39555/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.119:41327,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/39555/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Compute JJA anomaly for 1994-2023 w.r.t 1940-1969 for a lat-lon grid which bounds Colorado

In [11]:
%%time
tas_monthly_zarr      = xr.open_zarr(zarr_path +'e5_tas2m_monthly_1940_2023.zarr').VAR_2T
tas_monthly_zarr

CPU times: user 420 ms, sys: 74.8 ms, total: 495 ms
Wall time: 5.4 s


<xarray.DataArray 'VAR_2T' (time: 1009, latitude: 721, longitude: 1440)> Size: 4GB
dask.array<open_dataset-VAR_2T, shape=(1009, 721, 1440), dtype=float32, chunksize=(1000, 721, 180), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 8kB 1940-01-31 1940-02-29 ... 2024-01-31
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

In [12]:
tas_monthly_zarr      = tas_monthly_zarr.sel(latitude=slice(colorado_top,colorado_bottom),\
                                         longitude=slice(colorado_right,colorado_left))
tas_monthly_zarr 

<xarray.DataArray 'VAR_2T' (time: 1009, latitude: 17, longitude: 28)> Size: 2MB
dask.array<getitem, shape=(1009, 17, 28), dtype=float32, chunksize=(1000, 17, 28), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 136B 41.0 40.75 40.5 40.25 ... 37.5 37.25 37.0
  * longitude  (longitude) float64 224B 251.0 251.2 251.5 ... 257.2 257.5 257.8
  * time       (time) datetime64[ns] 8kB 1940-01-31 1940-02-29 ... 2024-01-31
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

In [13]:
# %%time
# tas_monthly_nc  = xr.open_mfdataset(zarr_path +'e5_tas2m_monthly_1940_2023.nc',engine='netcdf4',chunks='auto').VAR_2T
# tas_monthly_nc  = tas_monthly_nc.sel(latitude=slice(colorado_top,colorado_bottom),\
#                                          longitude=slice(colorado_right,colorado_left))
# tas_monthly_nc

#### Now compute monthly anomaly for JJA and comapre zarr vs netcdf

In [14]:
months = [6,7,8]

In [15]:
%%time
jja_tas_zarr = tas_monthly_zarr.sel(time=tas_monthly_zarr['time'].dt.month.isin(months))
# jja_tas_nc   = tas_monthly_nc.sel(time=tas_monthly_nc['time'].dt.month.isin(months))

CPU times: user 8.59 ms, sys: 71 µs, total: 8.66 ms
Wall time: 8.3 ms


In [16]:
%%time
baseline_period = np.arange(1940,1970)
current_period  = np.arange(1994,2024)
#
jja_baseline_zarr = jja_tas_zarr.sel(time = jja_tas_zarr['time'].dt.year.isin(baseline_period))
jja_current_zarr  = jja_tas_zarr.sel(time = jja_tas_zarr['time'].dt.year.isin(current_period))
#
# jja_baseline_nc   = jja_tas_nc.sel(time = jja_tas_nc['time'].dt.year.isin(baseline_period))
# jja_current_nc    = jja_tas_nc.sel(time = jja_tas_nc['time'].dt.year.isin(current_period))

CPU times: user 4.13 ms, sys: 0 ns, total: 4.13 ms
Wall time: 3.96 ms


In [17]:
#
jja_anomaly_zarr   = jja_current_zarr.mean('time') - jja_baseline_zarr.mean('time')
# jja_anomaly_nc     = jja_current_nc.mean('time') - jja_baseline_nc.mean('time')
# jja_anomaly_nc

In [18]:
# %%time
# ## Generate performance report
# with performance_report(filename ='zarr_jja_anomaly.html'):
#      jja_anomaly_zarr.to_dataset().to_zarr(zarr_path + 'e5_jja_anomaly.zarr',mode='w')

In [19]:
# %%time
# ## Generate performance report
# with performance_report(filename ='nc_jja_anomaly.html'):
#      jja_anomaly_nc.to_dataset().to_netcdf(zarr_path + 'e5_jja_anomaly.nc',mode='w')

## Plotting

In [ ]:
%%time
jja_anomaly_zarr = xr.open_zarr(zarr_path + 'e5_jja_anomaly.zarr').VAR_2T
jja_anomaly_nc   = xr.open_mfdataset(zarr_path + 'e5_jja_anomaly.nc').VAR_2T

In [ ]:
%%time
# Create a plot with a geographic projection
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))  # Using the Plate Carree projection

colorado_extent = [251, 258, 37, 41] 

# Set the extent to focus tightly on Colorado
ax.set_extent([colorado_right, colorado_left, colorado_bottom, colorado_top], crs=ccrs.PlateCarree(central_longitude=180))


# Add state borders, but only for Colorado
states = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='110m',
    facecolor='none')

ax.add_feature(states, edgecolor='black')  # Add only state borders
ax.add_feature(cfeature.BORDERS, linestyle=':', zorder=2)
ax.add_feature(cfeature.STATES, linestyle='-', edgecolor='black', zorder=3)

# Plot the data
jja_anomaly_zarr.plot(ax=ax, transform=ccrs.PlateCarree(central_longitude=180),
        x='longitude', y='latitude',
        cmap='viridis',  # Choose a colormap that suits your data
        add_colorbar=True)  # Optionally add a colorbar

#
# Add gridlines and labels 
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Display the plot
plt.show()

KeyboardInterrupt: 

In [ ]:
# Create a plot with a geographic projection
fig = plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))  # Using the Plate Carree projection

# Set the extent to focus tightly on Colorado
ax.set_extent([colorado_right, colorado_left, colorado_bottom, colorado_top], crs=ccrs.PlateCarree(central_longitude=180))

# Add state borders, but only for Colorado
states = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='110m',
    facecolor='none')

ax.add_feature(states, edgecolor='black')  # Add only state borders

# Plot the data
jja_anomaly_nc.plot(ax=ax, transform=ccrs.PlateCarree(central_longitude=180),
        x='longitude', y='latitude',
        cmap='viridis',  # Choose a colormap that suits your data
        add_colorbar=True)  # Optionally add a colorbar

ax.add_feature(cfeature.BORDERS, linestyle=':', zorder=2)
ax.add_feature(cfeature.STATES, linestyle='-', edgecolor='black', zorder=3)

#
# Add gridlines and labels 
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)

# Display the plot
plt.show()